# Tqdm audio progress ticker and Ricker for Jupyter notebooks

In [1]:
from tqdm_ricker import tqdm_audio_ticker, tqdm_audio_ricker # only import these once per session
import urllib
import os
import time

## Retro blip ticker

In [2]:
eg_link = "https://caspersci.uk.to/matryoshka.zip"
with tqdm_audio_ticker(unit='B', unit_scale=True, miniters=1,
              desc=eg_link.split('/')[-1]) as t:  # all optional kwargs
    urllib.request.urlretrieve(eg_link, filename=os.devnull,
                       reporthook=t.update_to, data=None)

matryoshka.zip:   0%|          | 33.0/260k [00:01<2:48:21, 25.7B/s] 


In [3]:
with tqdm_audio_ticker(["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n",
                      "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"],
                     freq_step=20) as pbar:
    for char in pbar:
        pbar.set_description("Processing %s" % char)
        time.sleep(0.1)

Processing z: 100%|██████████| 26/26 [00:02<00:00,  9.24it/s]


## Ricker -- *woo vu lubub dub dub*

In [4]:
eg_link = "https://caspersci.uk.to/matryoshka.zip"
with tqdm_audio_ricker(unit='B', unit_scale=True, miniters=1,
              desc=eg_link.split('/')[-1], total=50) as t:  # all optional kwargs
    urllib.request.urlretrieve(eg_link, filename=os.devnull,
                       reporthook=t.update_to, data=None)

matryoshka.zip:   0%|          | 33.0/260k [00:01<3:00:34, 24.0B/s] 


In [5]:
with tqdm_audio_ricker(["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n",
                      "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"],
                     total=26) as pbar:
    for char in pbar:
        pbar.set_description("Processing %s" % char)
        time.sleep(0.12)

Processing z: 100%|██████████| 26/26 [00:03<00:00,  7.78it/s]


In [6]:
with tqdm_audio_ticker(["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n",
                      "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"],
                     total=26, duration=0.1, freq_step=-50, start_freq=1500) as pbar:
    for char in pbar:
        pbar.set_description("Processing %s" % char)
        time.sleep(0.12)

Processing z: 100%|██████████| 26/26 [00:03<00:00,  7.76it/s]
